In [5]:
!pip install -q langchain langchain-community langchain-nomic faiss-cpu python-dotenv


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.7 MB/s eta 0:00:00


In [8]:
!pip install pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 35.4 MB/s eta 0:00:00


In [10]:
from google.colab import files
uploaded = files.upload()
pdf_path = next(iter(uploaded))


Saving Understanding_Climate_Change.pdf to Understanding_Climate_Change (1).pdf


In [11]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(pdf_path)
docs = loader.load()
print(f"Loaded {len(docs)} pages.")


Loaded 33 pages.


In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", "!", "?"]
)

chunks = splitter.split_documents(docs)
print(f"Created {len(chunks)} chunks.")



Created 170 chunks.


In [15]:
from langchain_nomic.embeddings import NomicEmbeddings
from langchain.vectorstores import FAISS
import os

os.environ["NOMIC_API_KEY"] = "nk-vXJD6f8Pl2RX7M2YADS7irz7-YBU-kZ96Gojpo05PaM"

embedding = NomicEmbeddings(model="nomic-embed-text-v1.5")

db = FAISS.from_documents(chunks, embedding)
retriever = db.as_retriever(search_kwargs={"k": 2})


In [17]:

!pip install -q langchain-groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.6 MB/s eta 0:00:00


In [18]:
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

os.environ["GROQ_API_KEY"] = "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"

llm = ChatGroq(
    model_name="llama3-8b-8192",
    groq_api_key=os.environ["GROQ_API_KEY"]
)

prompt = ChatPromptTemplate.from_template("""
Use the following context to answer the question.
If the answer is not in the context, say "I don't know".

Context:
{context}

Question: {question}
""")

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [19]:
while True:
    question = input("Ask a question (or 'exit'): ")
    if question.lower() == "exit":
        break
    answer = rag_chain.invoke(question)
    print("Answer:", answer)


Ask a question (or 'exit'): What are the consequences of rising global temperatures?
Answer: According to the context, the consequences of rising global temperatures are:

* Rising Temperatures
* Heatwaves

These consequences are mentioned in the page content of the first document, which is labeled as "Chapter 3: Effects of Climate Change".
Ask a question (or 'exit'): exit
